In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
strategy = tf.distribute.MirroredStrategy()

2024-10-25 04:30:57.595145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 04:30:57.618449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 04:30:57.618521: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 04:30:57.632476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-25 04:30:58.720660: W tensorflow/compiler/tf

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-10-25 04:31:01.135431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capability: 8.0
2024-10-25 04:31:01.137047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38380 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:4e:00.0, compute capability: 8.0


In [2]:
from data_loader.create_dataset import create_tf_datasets

from models.attention_unet import AttentionUNet3D
from models.unet import UNet3D

import numpy as np


In [3]:
train_dir = 'ImageSegmentation/BC/training_data'
test_dir = 'ImageSegmentation/BC/test_data'

train_dataset, val_dataset, test_dataset = create_tf_datasets(
    train_dir, 
    test_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=32
)

ImageSegmentation/BC/training_data/images/BC_Ch1.tif ImageSegmentation/BC/training_data/masks/BC_Ch1.tif
ImageSegmentation/BC/training_data/images/BC_G1.tif ImageSegmentation/BC/training_data/masks/BC_G1.tif
ImageSegmentation/BC/test_data/images/BC_Ch2.tif ImageSegmentation/BC/test_data/masks/BC_Ch2.tif
ImageSegmentation/BC/test_data/images/BC_G2.tif ImageSegmentation/BC/test_data/masks/BC_G2.tif


In [ ]:
x_train = np.array(train_dataset[0])
y_train = np.array(train_dataset[1])

x_val = np.array(val_dataset[0])
y_val = np.array(val_dataset[1])

with strategy.scope():
    unet = UNet3D().build_model()
    
    unet.compile(
        'adam',
        'dice'
    )

    unet.fit(
        x = x_train,
        y = y_train, 
        batch_size=8,
        epochs=50,
        validation_data = (x_val, y_val),
    )

Epoch 1/50


2024-10-25 04:32:07.855620: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-10-25 04:32:07.863899: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-10-25 04:32:09.095799: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.98GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 439/1556 ━━━━━━━━━━━━━━━━━━━━ 2:41 144ms/step - loss: 0.7661

In [ ]:
with strategy.scope():
    aunet = AttentionUNet3D().build_model()
    
    aunet.compile(
        'adam',
        'mse'
    )

    aunet.fit(
        x = x_train,
        y = y_train, 
        batch_size=8,
        epochs=50,
        validation_data = (x_val, y_val),
    )